# Basket Analysis with Association Rule and the Apriori Algorithm

Association rules analysis is a technique to uncover how items are associated to each other. There are 3 common ways to measure association:
+ Support - how popular an itemset is, as measured by proportion of transactions in which an itemset appears.
+ Confidence - how likely item Y is purchased when item X is purchased, expressed as {X -> Y}. This is measured by proportion of transactions with item X, in which item Y also appears.
+ Lift - how likely item Y is purchased when item X is purchased, while controlling for how popular item Y is. A lift value greater than 1 means that item Y is likely to be bought if item X is bought, while a value less than 1 means that item Y is unlikely to be bought if item X is bought.

MLxtend library, by Sebastian Raschka, has an implementation of the Apriori algorithm for extracting frequent item sets for further analysis. Apriori is an algorithm for frequent item set mining and association rule learning. It proceeds by identifying the frequent individual items and extending them to larger and larger item sets as long as those item sets appear sufficiently often in the database. The frequent item sets determined by Apriori can be used to determine association rules, which highlight general trends: this has applications in domains such as market basket analysis. [Source: https://en.wikipedia.org/wiki/Apriori_algorithm] 

**Data:**      

Instacart, a grocery ordering and delivery app, aims to make it easy to fill your refrigerator and pantry with your personal favorites and staples when you need them.

Anonymised data from Instacart on customer orders over time, accessed from Kaggle through the following link: https://www.kaggle.com/c/instacart-market-basket-analysis/data

The dataset is a relational set of files describing customers' orders over time. The dataset contains a sample of over 3 million grocery orders from more than 200,000 Instacart users. For each user, there between 4 and 100 of their orders, with the sequence of products purchased in each order.

In [1]:
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

import os

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

## 1. Load the data

In [2]:
aisles = pd.read_csv('aisles.csv')
dept = pd.read_csv('departments.csv')
order_prior = pd.read_csv('order_products__prior.csv')
order_train = pd.read_csv('order_products__train.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

In [3]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [4]:
dept.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [5]:
order_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [6]:
order_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [7]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [8]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


## 2. Merge tables

### 2.1 Department, Aisle and Products

In [9]:
merge1 = pd.merge(products, dept, on='department_id')
merge1 = pd.merge(merge1, aisles, on='aisle_id')
merge1.head()

,product_id,product_name,aisle_id,department_id,department,aisle
0,1,Chocolate Sandwich Cookies,61,19,snacks,cookies cakes
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,snacks,cookies cakes
2,102,Danish Butter Cookies,61,19,snacks,cookies cakes
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,snacks,cookies cakes
4,285,Mini Nilla Wafers Munch Pack,61,19,snacks,cookies cakes


### 2.2 Order_products_prior,  Order_products__train and Orders

In [10]:
append_orders = order_prior.append(order_train)
append_orders.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [11]:
merge2 = pd.merge(orders, append_orders, on='order_id')
merge2.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


### 2.3 Merge all tables

In [12]:
data = pd.merge(merge1, merge2, on='product_id')

In [13]:
data.head()

,product_id,product_name,aisle_id,department_id,department,aisle,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered
0,1,Chocolate Sandwich Cookies,61,19,snacks,cookies cakes,3139998,138,prior,28,6,11,3.0,5,0
1,1,Chocolate Sandwich Cookies,61,19,snacks,cookies cakes,1977647,138,prior,30,6,17,20.0,1,1
2,1,Chocolate Sandwich Cookies,61,19,snacks,cookies cakes,389851,709,prior,2,0,21,6.0,20,0
3,1,Chocolate Sandwich Cookies,61,19,snacks,cookies cakes,63770,751,train,7,2,6,30.0,4,0
4,1,Chocolate Sandwich Cookies,61,19,snacks,cookies cakes,652770,764,prior,1,3,13,NaN,10,0


## 3. Prepare Data for Analysis

In [14]:
df = data[['order_id','product_name']]
df.head()

,order_id,product_name
0,3139998,Chocolate Sandwich Cookies
1,1977647,Chocolate Sandwich Cookies
2,389851,Chocolate Sandwich Cookies
3,63770,Chocolate Sandwich Cookies
4,652770,Chocolate Sandwich Cookies


In [15]:
df.sort_values(by=['order_id'], inplace=True)
df.head()

,order_id,product_name
27112097,1,Bag of Organic Bananas
12640048,1,Organic Whole String Cheese
29726800,1,Organic Hass Avocado
13244505,1,Organic 4% Milk Fat Whole Milk Cottage Cheese
26385801,1,Cucumber Kirby


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33819106 entries, 27112097 to 284591
Data columns (total 2 columns):
order_id        int64
product_name    object
dtypes: int64(1), object(1)
memory usage: 774.1+ MB


## 4. Retrieve Data for Household Products

For manageability, runtime and computing power reasons, only sales of household products will be used. Household products are department 17.

In [17]:
#load products
products = pd.read_csv('products.csv')

#drop irrelevant columns
products.drop(['product_id', 'aisle_id'], axis = 1, inplace=True)

#keep only household products (dept 17)
household = products[products['department_id'] == 17]
household.head()

,product_name,department_id
13,Fresh Scent Dishwasher Cleaner,17
47,"School Glue, Washable, No Run",17
56,Flat Toothpicks,17
70,Ultra 7 Inch Polypropylene Traditional Plates,17
104,"Easy Grab 9\""x13\"" Oblong Glass Bakeware",17


In [18]:
#create empty list
household_list = []

#add product names to list
for i, rows in household.iterrows():
    i = rows.product_name
    household_list.append(i)

In [19]:
#remove products not household product 
df_household = df[df['product_name'].isin(household_list)]
df_household.head()

,order_id,product_name
16935322,5,Everyday Facial Tissues
16938618,5,Sensitive Toilet Paper
16927700,5,One Ply Choose A Size Big Roll Paper Towel Rolls
16684866,6,Clean Day Lavender Scent Room Freshener Spray
16418408,6,Dryer Sheets Geranium Scent


In [20]:
df_household.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 774652 entries, 16935322 to 16287309
Data columns (total 2 columns):
order_id        774652 non-null int64
product_name    774652 non-null object
dtypes: int64(1), object(1)
memory usage: 17.7+ MB


In [21]:
#create basket df from df_household using custom aggregations
basket = df_household.groupby('order_id', as_index = False).agg({'product_name': lambda x: list(x)})
basket = basket.set_index('order_id')
basket.head()

,product_name
order_id,
5,"[Everyday Facial Tissues, Sensitive Toilet Pa..."
6,[Clean Day Lavender Scent Room Freshener Spray...
18,[100% Recycled Paper Towels]
22,[Sandwich Bags]
27,[Parchment Paper]


Association Analysis requires that all data for a transaction be included in a single row, and that items are one-hot encoded.      

To one-hot encode product names, MultiLabelBinarizer will be used.

In [22]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
basket = basket.join(pd.DataFrame(mlb.fit_transform(basket.pop('product_name')),
                                  columns = mlb.classes_,
                                  index = basket.index))
basket.head()

,#2 Mechanical Pencils,1 Cup Measuring Cup,1 Ply Napkins,1 Ply Paper Towels,1 Ply White Luncheon Napkins,10 Inch Wheat Straw Plates,10 oz Paper Bowls,"10.25\"" Cast Iron Skillet","10.25\"" Elegant Fluted Party Plates",100 % Recycled Paper Towels,...,Zipper Sandwich Bags,e-Tronic Wipes,"flings! Laundry Detergent Pacs, Original, 57 Count Laundry",flings! Original Laundry Detergent Pacs,iChef Casserole Pans with Lids (10 7/16 in x 8 in x 1 3/4 in),with Bleach Disinfectant Cleanser Scratch Free Lavender Fresh,with Bleach Powder Cleanser,with Color Safe Brightener Power Paks 2in1 Stain Fighter,with Dawn Action Pacs Fresh Scent Dishwasher Detergent Pacs,with Twist Ties Sandwich & Storage Bags
order_id,,,,,,,,,,,,,,,,,,,,,
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Frequent item combinations that have a support of at least 0.1% can now be generated using the apriori from mlxtend.

In [23]:
frequent_item_combos = apriori(basket, min_support = 0.001, use_colnames = True)
frequent_item_combos.head()

,support,itemsets
0,0.005759,(1 Ply Paper Towels)
1,0.007843,(100% Recycled 2 Ply Jumbo Paper Towel Roll)
2,0.003026,(100% Recycled 2 Ply Paper Towels)
3,0.010471,(100% Recycled Bath Tissue Rolls)
4,0.015324,(100% Recycled Bathroom Tissue)


Finally, generate the rules with their corresponding support, confidence and lift.

In [24]:
rules = association_rules(frequent_item_combos, metric = "lift", min_threshold = 1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(100% Recycled Bath Tissue Rolls),(100% Recycled Paper Towels),0.010471,0.058987,0.002236,0.213538,3.620053,0.001618,1.196513
1,(100% Recycled Paper Towels),(100% Recycled Bath Tissue Rolls),0.058987,0.010471,0.002236,0.037904,3.620053,0.001618,1.028514
2,(100% Recycled Bathroom Tissue),(100% Recycled Paper Towels),0.015324,0.058987,0.001413,0.092234,1.563626,0.000509,1.036625
3,(100% Recycled Paper Towels),(100% Recycled Bathroom Tissue),0.058987,0.015324,0.001413,0.023961,1.563626,0.000509,1.008849
4,(100% Recycled Bathroom Tissue),(2-Ply Right Size 100% Recycled Paper Towels),0.015324,0.007244,0.001060,0.069176,9.549760,0.000949,1.066535


#### Interpreting the table above 
+ **Antecedent (A)**
+ **Consequency (C)**
+ **Support:** defined for itemsets, not association rules. The table produced by the association rule mining algorithm contains three different support metrics: 
    + 'antecedent support' - computes proportion of transactions that contain the antecedent A
    + 'consequent support' - computes the support for the itemset of the consequent C
    + 'support' - computes the support of the combined itemset (A and C). Thus, 'support' depends on 'antecedent support' and 'consequent support' via min('antecedent support', 'consequent support').
    Typically, support is used to measure the abundance or frequency (often interpreted as significance or importance) of an itemset in a database. We refer to an itemset as a "frequent itemset" if you support is larger than a specified minimum-support threshold. Note that in general, due to the downward closure property, all subsets of a frequent itemset are also frequent.
+ **Confidence:** the confidence of a rule A->C is the probability of seeing the consequent in a transaction given that it also contains the antecedent. Note that the metric is not symmetric or directed; for instance, the confidence for A->C is different than the confidence for C->A. The confidence is 1 (maximal) for a rule A->C if the consequent and antecedent always occur together. 
+ **Lift:** is commonly used to measure how much more often the antecedent and consequent of a rule A->C occur together than we would expect if they were statistically independent. If A and C are independent, the Lift score will be exactly 1.   
+ **Leverage:** computes the difference between the observed frequency of A and C appearing together and the frequency that would be expected if A and C were independent. An leverage value of 0 indicates independence.
+ **Conviction:** a high conviction value means that the consequent is highly dependent on the antecedent. For instance, in the case of a perfect confidence score, the denominator becomes 0 (due to 1 - 1) for which the conviction score is defined as 'inf'. Similar to lift, if items are independent, the conviction is 1.

[Source: http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/]